In [7]:
"""
Smart Campus Navigation and Rescue Planner
KIIT Campus-25 Emergency Navigation System

Movement Cost Explanation:
  1 = Corridor, Open Ground (Fast, clear paths during emergency)
  2 = Lift, Hostel (Moderate - elevators slow, hostel has doors)
  3 = Stairs (Slow, physical effort, bottleneck risk)
  4 = Canteen (Very crowded, difficult to navigate)
  ∞ = Blocked (Construction/Crowd/Fire - Impassable)
"""

import random
import time
from collections import deque
import heapq

# Cell type definitions with movement costs and display symbols
# Each cell type has: cost (time to traverse), symbol (for display), name (description)
CELL_TYPES = {
    'corridor': {'cost': 1, 'symbol': 'C', 'name': 'Corridor'},
    'stairs': {'cost': 3, 'symbol': 'S', 'name': 'Stairs'},
    'lift': {'cost': 2, 'symbol': 'L', 'name': 'Lift'},
    'ground': {'cost': 1, 'symbol': 'G', 'name': 'Open Ground'},
    'hostel': {'cost': 2, 'symbol': 'H', 'name': 'Hostel'},
    'canteen': {'cost': 4, 'symbol': 'T', 'name': 'Canteen'},
    'blocked': {'cost': float('inf'), 'symbol': 'X', 'name': 'Blocked'}
}

class CampusMap:
    """Represents KIIT Campus-25 grid map"""
    
    def __init__(self, rows=15, cols=15):
        self.rows = rows
        self.cols = cols
        self.grid = []
        self.costs = []
        self.start = None
        self.goals = []
        
    def generate_random_map(self):
        """Generate a completely random campus map with diverse terrain"""
        # Initialize grid with random cell types
        cell_options = ['corridor', 'stairs', 'lift', 'ground', 'hostel', 'canteen']
        
        for i in range(self.rows):
            row = []
            cost_row = []
            for j in range(self.cols):
                # Randomly assign cell type
                cell_type = random.choice(cell_options)
                row.append(cell_type)
                cost_row.append(CELL_TYPES[cell_type]['cost'])
            self.grid.append(row)
            self.costs.append(cost_row)
        
        # Block 20-30% of cells randomly to simulate construction/crowd
        total_cells = self.rows * self.cols
        block_count = random.randint(int(0.20 * total_cells), int(0.30 * total_cells))
        
        blocked_positions = set()
        while len(blocked_positions) < block_count:
            r = random.randint(0, self.rows - 1)
            c = random.randint(0, self.cols - 1)
            blocked_positions.add((r, c))
        
        # Mark blocked cells as impassable
        for r, c in blocked_positions:
            self.grid[r][c] = 'blocked'
            self.costs[r][c] = float('inf')
        
        # Set start position (ensure not blocked)
        while True:
            self.start = (random.randint(0, self.rows - 1), random.randint(0, self.cols - 1))
            if self.grid[self.start[0]][self.start[1]] != 'blocked':
                break
        
        # Set multiple goal positions (2-4 safe zones)
        goal_count = random.randint(2, 4)
        while len(self.goals) < goal_count:
            goal = (random.randint(0, self.rows - 1), random.randint(0, self.cols - 1))
            if (self.grid[goal[0]][goal[1]] != 'blocked' and 
                goal != self.start and 
                goal not in self.goals):
                self.goals.append(goal)
    
    def print_grid(self, path=None, explored=None, title="CAMPUS MAP"):
        """Print grid as matrix in console"""
        print("\n" + "="*80)
        print(f"{title}")
        print("="*80)
        
        # Print legend
        print("\nLEGEND:")
        for key, value in CELL_TYPES.items():
            if key != 'blocked':
                print(f"  {value['symbol']} = {value['name']} (Cost: {value['cost']})")
        print(f"  X = Blocked (Impassable)")
        print(f"  * = Path | @ = Explored | S = Start | # = Goal")
        print()
        
        # Create path and explored sets for quick lookup
        path_set = set(path) if path else set()
        explored_set = set(explored) if explored else set()
        goal_set = set(self.goals)
        
        # Print column headers
        print("     ", end="")
        for c in range(self.cols):
            print(f"{c:3}", end="")
        print()
        print("   " + "-" * (self.cols * 3 + 2))
        
        # Print grid
        for r in range(self.rows):
            print(f"{r:2} | ", end="")
            for c in range(self.cols):
                pos = (r, c)
                
                if pos == self.start:
                    print(" S ", end="")
                elif pos in goal_set:
                    print(" # ", end="")
                elif pos in path_set:
                    print(" * ", end="")
                elif pos in explored_set:
                    print(" @ ", end="")
                else:
                    symbol = CELL_TYPES[self.grid[r][c]]['symbol']
                    print(f" {symbol} ", end="")
            print()
        print("="*80)
    
    def get_neighbors(self, pos):
        """Get valid neighboring cells (4-directional)"""
        r, c = pos
        neighbors = []
        
        # Check all 4 directions: up, down, left, right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for dr, dc in directions:
            new_r, new_c = r + dr, c + dc
            
            # Check bounds and if not blocked
            if (0 <= new_r < self.rows and 
                0 <= new_c < self.cols and 
                self.grid[new_r][new_c] != 'blocked'):
                neighbors.append((new_r, new_c))
        
        return neighbors
    
    def get_cost(self, pos):
        """Get movement cost for a position"""
        return self.costs[pos[0]][pos[1]]


class PathfindingAlgorithms:
    """Implementation of all 4 search algorithms from scratch"""
    
    def __init__(self, campus_map):
        self.map = campus_map
    
    # === HEURISTIC FUNCTIONS ===
    
    def manhattan_distance(self, pos1, pos2):
        """
        Standard Manhattan distance heuristic for grid-based navigation
        Formula: |x1 - x2| + |y1 - y2|
        Admissible: Never overestimates the actual cost
        """
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def custom_heuristic(self, pos1, pos2):
        """
        Custom terrain-aware heuristic for emergency navigation
        Combines Manhattan distance with terrain difficulty penalties
        Helps avoid dangerous/slow areas during evacuation
        """
        base_distance = abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
        
        # Add penalty based on current cell type
        cell_type = self.map.grid[pos1[0]][pos1[1]]
        
        # Heavy penalty for stairs and canteen during emergency
        if cell_type == 'stairs':
            penalty = 3  # Stairs are slow and dangerous in emergency
        elif cell_type == 'canteen':
            penalty = 2  # Canteen is crowded
        elif cell_type == 'hostel':
            penalty = 1  # Hostel has doors/obstacles
        else:
            penalty = 0
        
        return base_distance + penalty
    
    def find_nearest_goal(self, pos, goals, heuristic_func):
        """Find the closest goal from current position"""
        min_dist = float('inf')
        nearest = goals[0]
        
        for goal in goals:
            dist = heuristic_func(pos, goal)
            if dist < min_dist:
                min_dist = dist
                nearest = goal
        
        return nearest
    
    # === ALGORITHM 1: BFS ===
    
    def bfs(self):
        """
        Breadth-First Search - Uninformed search algorithm
        Uses FIFO queue to explore nodes level by level
        Guarantees shortest path in terms of number of steps
        Time Complexity: O(V + E) where V=vertices, E=edges
        """
        start_time = time.time()
        
        # Initialize queue with start position and path
        queue = deque([(self.map.start, [self.map.start], 0)])
        visited = {self.map.start}
        explored_order = []
        
        while queue:
            current, path, cost = queue.popleft()
            explored_order.append(current)
            
            # Check if goal reached
            if current in self.map.goals:
                end_time = time.time()
                actual_cost = self.calculate_path_cost(path)
                return {
                    'path': path,
                    'cost': actual_cost,
                    'nodes_explored': len(explored_order),
                    'time': (end_time - start_time) * 1000,
                    'explored': explored_order
                }
            
            # Explore all neighbors
            for neighbor in self.map.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_path = path + [neighbor]
                    queue.append((neighbor, new_path, 0))
        
        # No path found
        end_time = time.time()
        return {
            'path': None,
            'cost': float('inf'),
            'nodes_explored': len(explored_order),
            'time': (end_time - start_time) * 1000,
            'explored': explored_order
        }
    
    # === ALGORITHM 2: DFS ===
    
    def dfs(self):
        """
        Depth-First Search - Uninformed search algorithm
        Uses LIFO stack to explore deeply before backtracking
        Fast but may not find optimal path
        Time Complexity: O(V + E) where V=vertices, E=edges
        """
        start_time = time.time()
        
        # Initialize stack with start position and path
        stack = [(self.map.start, [self.map.start], 0)]
        visited = {self.map.start}
        explored_order = []
        
        while stack:
            current, path, cost = stack.pop()
            explored_order.append(current)
            
            # Check if goal reached
            if current in self.map.goals:
                end_time = time.time()
                actual_cost = self.calculate_path_cost(path)
                return {
                    'path': path,
                    'cost': actual_cost,
                    'nodes_explored': len(explored_order),
                    'time': (end_time - start_time) * 1000,
                    'explored': explored_order
                }
            
            # Explore neighbors (reversed to maintain consistent order)
            neighbors = self.map.get_neighbors(current)
            for neighbor in reversed(neighbors):
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_path = path + [neighbor]
                    stack.append((neighbor, new_path, 0))
        
        # No path found
        end_time = time.time()
        return {
            'path': None,
            'cost': float('inf'),
            'nodes_explored': len(explored_order),
            'time': (end_time - start_time) * 1000,
            'explored': explored_order
        }
    
    # === ALGORITHM 3: Best First Search ===
    
    def best_first_search(self, heuristic='manhattan'):
        """
        Best First Search - Greedy informed search algorithm
        Uses priority queue ordered by heuristic value only
        Fast and goal-directed but not guaranteed optimal
        Time Complexity: O(E log V) where V=vertices, E=edges
        """
        start_time = time.time()
        
        # Select heuristic function
        heuristic_func = self.manhattan_distance if heuristic == 'manhattan' else self.custom_heuristic
        
        # Initialize priority queue with heuristic value
        nearest_goal = self.find_nearest_goal(self.map.start, self.map.goals, heuristic_func)
        h = heuristic_func(self.map.start, nearest_goal)
        
        priority_queue = [(h, self.map.start, [self.map.start])]
        visited = {self.map.start}
        explored_order = []
        
        while priority_queue:
            _, current, path = heapq.heappop(priority_queue)
            explored_order.append(current)
            
            # Check if goal reached
            if current in self.map.goals:
                end_time = time.time()
                actual_cost = self.calculate_path_cost(path)
                return {
                    'path': path,
                    'cost': actual_cost,
                    'nodes_explored': len(explored_order),
                    'time': (end_time - start_time) * 1000,
                    'explored': explored_order
                }
            
            # Explore neighbors using heuristic
            for neighbor in self.map.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    nearest_goal = self.find_nearest_goal(neighbor, self.map.goals, heuristic_func)
                    h = heuristic_func(neighbor, nearest_goal)
                    new_path = path + [neighbor]
                    heapq.heappush(priority_queue, (h, neighbor, new_path))
        
        # No path found
        end_time = time.time()
        return {
            'path': None,
            'cost': float('inf'),
            'nodes_explored': len(explored_order),
            'time': (end_time - start_time) * 1000,
            'explored': explored_order
        }
    
    # === ALGORITHM 4: A* Search ===
    
    def astar(self, heuristic='manhattan'):
        """
        A* Search - Optimal informed search algorithm
        Uses priority queue ordered by f(n) = g(n) + h(n)
        g(n) = actual cost from start, h(n) = heuristic to goal
        Guarantees optimal path if heuristic is admissible
        Time Complexity: O(E log V) where V=vertices, E=edges
        """
        start_time = time.time()
        
        # Select heuristic function
        heuristic_func = self.manhattan_distance if heuristic == 'manhattan' else self.custom_heuristic
        
        # Initialize priority queue with f-score
        nearest_goal = self.find_nearest_goal(self.map.start, self.map.goals, heuristic_func)
        h = heuristic_func(self.map.start, nearest_goal)
        f = 0 + h  # f = g + h
        
        priority_queue = [(f, 0, self.map.start, [self.map.start])]
        g_scores = {self.map.start: 0}  # Track best g-score to each node
        explored_order = []
        
        while priority_queue:
            _, g, current, path = heapq.heappop(priority_queue)
            
            # Skip if we've already found a better path to this node
            if current in g_scores and g > g_scores[current]:
                continue
            
            explored_order.append(current)
            
            # Check if goal reached
            if current in self.map.goals:
                end_time = time.time()
                return {
                    'path': path,
                    'cost': g,  # Return actual cost (g-score)
                    'nodes_explored': len(explored_order),
                    'time': (end_time - start_time) * 1000,
                    'explored': explored_order
                }
            
            # Explore neighbors with cost awareness
            for neighbor in self.map.get_neighbors(current):
                new_g = g + self.map.get_cost(neighbor)  # Add actual movement cost
                
                # Only proceed if this is a better path
                if neighbor not in g_scores or new_g < g_scores[neighbor]:
                    g_scores[neighbor] = new_g
                    nearest_goal = self.find_nearest_goal(neighbor, self.map.goals, heuristic_func)
                    h = heuristic_func(neighbor, nearest_goal)
                    f = new_g + h  # f = g + h
                    new_path = path + [neighbor]
                    heapq.heappush(priority_queue, (f, new_g, neighbor, new_path))
        
        # No path found
        end_time = time.time()
        return {
            'path': None,
            'cost': float('inf'),
            'nodes_explored': len(explored_order),
            'time': (end_time - start_time) * 1000,
            'explored': explored_order
        }
    
    def calculate_path_cost(self, path):
        """Calculate total cost of a path"""
        if not path:
            return float('inf')
        
        total_cost = 0
        for pos in path:
            total_cost += self.map.get_cost(pos)
        
        return total_cost


def print_comparison_table(results):
    """Print comparison table of all algorithms"""
    print("\n" + "="*100)
    print("ALGORITHM COMPARISON TABLE")
    print("="*100)
    
    header = f"{'Algorithm':<25} {'Path Length':<15} {'Path Cost':<12} {'Nodes':<10} {'Time (ms)':<12} {'Optimal':<10}"
    print(header)
    print("-"*100)
    
    for algo_name, result in results.items():
        path_len = len(result['path']) if result['path'] else 'N/A'
        cost = f"{result['cost']:.2f}" if result['cost'] != float('inf') else 'N/A'
        nodes = result['nodes_explored']
        time_ms = f"{result['time']:.4f}"
        optimal = "Yes" if result['cost'] != float('inf') and result['path'] else "No"
        
        row = f"{algo_name:<25} {str(path_len):<15} {cost:<12} {nodes:<10} {time_ms:<12} {optimal:<10}"
        print(row)
    
    print("="*100)


def dynamic_obstacle_test(campus_map, pathfinder, original_path):
    """
    Simulate dynamic obstacles appearing during navigation
    Randomly blocks 3-5 cells on the original path to test algorithm adaptation
    This simulates real emergency scenarios where new obstacles appear
    """
    print("\n" + "="*80)
    print("DYNAMIC OBSTACLE TEST")
    print("="*80)
    
    if not original_path or len(original_path) < 5:
        print("Path too short for dynamic testing!")
        return None
    
    # Block 3-5 random cells from the path (excluding start and goal)
    num_blocks = random.randint(3, min(5, len(original_path) - 2))
    blockable = original_path[1:-1]  # Exclude start and goal
    
    if len(blockable) < num_blocks:
        num_blocks = len(blockable)
    
    blocked_cells = random.sample(blockable, num_blocks)
    
    print(f"\nBlocking {num_blocks} cells on the original path:")
    for cell in blocked_cells:
        r, c = cell
        print(f"  Position ({r}, {c}): {campus_map.grid[r][c]} → BLOCKED")
        campus_map.grid[r][c] = 'blocked'
        campus_map.costs[r][c] = float('inf')
    
    print("\nRe-running all algorithms with new obstacles...")
    return run_all_algorithms(campus_map, pathfinder, dynamic=True)


def run_all_algorithms(campus_map, pathfinder, dynamic=False):
    """Run all algorithms and return results"""
    algorithms = {
        'BFS': pathfinder.bfs,
        'DFS': pathfinder.dfs,
        'Best First (Manhattan)': lambda: pathfinder.best_first_search('manhattan'),
        'Best First (Custom)': lambda: pathfinder.best_first_search('custom'),
        'A* (Manhattan)': lambda: pathfinder.astar('manhattan'),
        'A* (Custom)': lambda: pathfinder.astar('custom')
    }
    
    results = {}
    
    prefix = "[DYNAMIC] " if dynamic else ""
    
    for algo_name, algo_func in algorithms.items():
        print(f"\n{prefix}Running {algo_name}...")
        result = algo_func()
        results[algo_name] = result
        
        if result['path']:
            print(f"  ✓ Path found: {len(result['path'])} steps, Cost: {result['cost']:.2f}")
            print(f"  ✓ Nodes explored: {result['nodes_explored']}, Time: {result['time']:.4f}ms")
        else:
            print(f"  ✗ No path found")
            print(f"  ✓ Nodes explored: {result['nodes_explored']}, Time: {result['time']:.4f}ms")
    
    return results


def analyze_results(initial_results, dynamic_results=None):
    """Analyze and provide conclusion"""
    print("\n" + "="*80)
    print("ANALYSIS & CONCLUSION")
    print("="*80)
    
    # Find optimal algorithm(s)
    valid_results = {name: res for name, res in initial_results.items() if res['path']}
    
    if not valid_results:
        print("\nNo valid paths found!")
        return
    
    min_cost = min(res['cost'] for res in valid_results.values())
    optimal_algos = [name for name, res in valid_results.items() if res['cost'] == min_cost]
    
    print(f"\n1. OPTIMALITY:")
    print(f"   Minimum cost found: {min_cost:.2f}")
    print(f"   Optimal algorithm(s): {', '.join(optimal_algos)}")
    
    # Efficiency comparison
    print(f"\n2. EFFICIENCY (Node Exploration):")
    for name in optimal_algos:
        nodes = initial_results[name]['nodes_explored']
        print(f"   {name}: {nodes} nodes")
    
    # Speed comparison
    print(f"\n3. EXECUTION SPEED:")
    for name in optimal_algos:
        time_ms = initial_results[name]['time']
        print(f"   {name}: {time_ms:.4f} ms")
    
    # Path length
    print(f"\n4. PATH LENGTH:")
    for name in optimal_algos:
        length = len(initial_results[name]['path'])
        print(f"   {name}: {length} steps")
    
    # Dynamic adaptation
    if dynamic_results:
        print(f"\n5. DYNAMIC ADAPTATION:")
        for name in optimal_algos:
            if dynamic_results[name]['path']:
                new_cost = dynamic_results[name]['cost']
                cost_change = new_cost - min_cost
                print(f"   {name}: New cost {new_cost:.2f} (+{cost_change:.2f})")
            else:
                print(f"   {name}: Failed to find new path")
    
    # Final recommendation
    print(f"\n" + "="*80)
    print("FINAL RECOMMENDATION:")
    print("="*80)
    
    # Choose A* Manhattan as best if it's optimal
    if 'A* (Manhattan)' in optimal_algos:
        winner = 'A* (Manhattan)'
    else:
        winner = optimal_algos[0]
    
    print(f"\n🏆 BEST ALGORITHM: {winner}")
    print(f"\nREASONS:")
    print(f"  1. Finds optimal path with lowest cost ({min_cost:.2f})")
    print(f"  2. Efficient node exploration")
    print(f"  3. Fast execution time")
    print(f"  4. Adapts to dynamic obstacles")
    print(f"  5. Guaranteed optimality (uses both cost and heuristic)")
    
    print(f"\nWHY OTHERS FALL SHORT:")
    if 'BFS' in initial_results:
        bfs_nodes = initial_results['BFS']['nodes_explored']
        winner_nodes = initial_results[winner]['nodes_explored']
        if bfs_nodes > winner_nodes:
            print(f"  - BFS: Explores {bfs_nodes - winner_nodes} more nodes (less efficient)")
    
    if 'DFS' in valid_results:
        dfs_cost = initial_results['DFS']['cost']
        if dfs_cost > min_cost:
            print(f"  - DFS: Suboptimal cost {dfs_cost:.2f} (not reliable)")
    
    print(f"\nFor emergency campus navigation, {winner} is the clear choice!")
    print("="*80)


def main():
    """
    Main execution function for Smart Campus Navigation System
    Steps:
    1. Generate random campus map with obstacles
    2. Initialize pathfinding algorithms
    3. Run all 6 algorithms (BFS, DFS, Best First×2, A*×2)
    4. Visualize optimal path
    5. Test dynamic obstacle adaptation
    6. Analyze and provide conclusion
    """
    print("="*80)
    print("SMART CAMPUS NAVIGATION AND RESCUE PLANNER")
    print("KIIT Campus-25 Emergency Pathfinding System")
    print("="*80)
    
    # Step 1: Generate campus map
    print("\n[STEP 1] Generating Random Campus Map...")
    campus = CampusMap(rows=15, cols=15)
    campus.generate_random_map()
    
    print(f"\nCampus Grid: {campus.rows}x{campus.cols}")
    print(f"Start Position: {campus.start}")
    print(f"Goal Positions (Safe Zones): {campus.goals}")
    
    blocked_count = sum(1 for row in campus.grid for cell in row if cell == 'blocked')
    blocked_pct = (blocked_count / (campus.rows * campus.cols)) * 100
    print(f"Blocked Cells: {blocked_count} ({blocked_pct:.1f}%)")
    
    campus.print_grid(title="INITIAL CAMPUS MAP")
    
    # Step 2: Initialize pathfinding
    print("\n[STEP 2] Initializing Pathfinding Algorithms...")
    print("\nAlgorithms to be tested:")
    print("  1. BFS (Breadth-First Search) - Uninformed, Complete")
    print("  2. DFS (Depth-First Search) - Uninformed, Fast")
    print("  3. Best First Search (Manhattan) - Informed, Greedy")
    print("  4. Best First Search (Custom) - Informed, Terrain-Aware")
    print("  5. A* (Manhattan) - Optimal, Efficient")
    print("  6. A* (Custom) - Optimal, Terrain-Aware")
    
    print("\nHeuristic Functions:")
    print("  • Manhattan Distance: h(n) = |x1-x2| + |y1-y2|")
    print("  • Custom Heuristic: Manhattan + Terrain Penalty")
    print("    - Stairs penalty: +3 (slow, dangerous in emergency)")
    print("    - Canteen penalty: +2 (crowded)")
    print("    - Hostel penalty: +1 (doors, obstacles)")
    
    pathfinder = PathfindingAlgorithms(campus)
    print("\n✓ Pathfinding system initialized successfully!")
    
    # Step 3: Run all algorithms
    print("\n[STEP 3] Running All Search Algorithms...")
    print("-"*80)
    
    initial_results = run_all_algorithms(campus, pathfinder)
    
    # Step 4: Print comparison
    print_comparison_table(initial_results)
    
    # Step 5: Visualize best path
    print("\n[STEP 4] Visualizing Optimal Path...")
    best_algo = min(
        ((name, res) for name, res in initial_results.items() if res['path']),
        key=lambda x: x[1]['cost'],
        default=(None, None)
    )
    
    if best_algo[0]:
        campus.print_grid(
            path=best_algo[1]['path'],
            explored=best_algo[1]['explored'],
            title=f"OPTIMAL PATH: {best_algo[0]}"
        )
    
    # Step 6: Dynamic obstacle testing
    print("\n[STEP 5] Testing Dynamic Obstacle Adaptation...")
    
    if best_algo[0] and best_algo[1]['path']:
        dynamic_results = dynamic_obstacle_test(campus, pathfinder, best_algo[1]['path'])
        
        if dynamic_results:
            campus.print_grid(title="CAMPUS MAP WITH NEW OBSTACLES")
            print_comparison_table(dynamic_results)
            
            # Visualize new path
            new_best = min(
                ((name, res) for name, res in dynamic_results.items() if res['path']),
                key=lambda x: x[1]['cost'],
                default=(None, None)
            )
            
            if new_best[0]:
                campus.print_grid(
                    path=new_best[1]['path'],
                    explored=new_best[1]['explored'],
                    title=f"NEW OPTIMAL PATH: {new_best[0]}"
                )
            
            # Final analysis
            analyze_results(initial_results, dynamic_results)
        else:
            analyze_results(initial_results)
    else:
        analyze_results(initial_results)
    
    print("\n" + "="*80)
    print("ASSIGNMENT COMPLETED SUCCESSFULLY!")
    print("="*80)


if __name__ == "__main__":
    # Run with random seed for unique results every time
    # Remove random.seed() line to get truly random maps
    random.seed()
    main()

SMART CAMPUS NAVIGATION AND RESCUE PLANNER
KIIT Campus-25 Emergency Pathfinding System

[STEP 1] Generating Random Campus Map...

Campus Grid: 15x15
Start Position: (2, 4)
Goal Positions (Safe Zones): [(12, 11), (14, 0), (0, 1)]
Blocked Cells: 63 (28.0%)

INITIAL CAMPUS MAP

LEGEND:
  C = Corridor (Cost: 1)
  S = Stairs (Cost: 3)
  L = Lift (Cost: 2)
  G = Open Ground (Cost: 1)
  H = Hostel (Cost: 2)
  T = Canteen (Cost: 4)
  X = Blocked (Impassable)
  * = Path | @ = Explored | S = Start | # = Goal

       0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   -----------------------------------------------
 0 |  L  #  L  H  X  X  C  T  X  H  X  C  H  G  X 
 1 |  G  X  X  X  C  X  X  C  X  X  L  H  L  T  G 
 2 |  X  T  X  L  S  S  H  X  H  X  C  S  L  C  T 
 3 |  X  G  X  X  S  G  L  X  L  X  S  S  X  L  L 
 4 |  X  G  L  S  H  S  C  G  L  H  T  H  C  S  L 
 5 |  X  H  H  C  H  X  L  T  T  L  X  X  L  C  X 
 6 |  C  T  L  L  H  T  H  C  H  X  C  G  X  C  X 
 7 |  L  X  H  S  C  X  L  X  H  S  